In [1]:
import pickle
import pycaret
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tsfresh import select_features
from pycaret.classification import *
from tsfresh import extract_features
from tsfresh import extract_relevant_features
from sklearn.ensemble import ExtraTreesClassifier
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import MinimalFCParameters, EfficientFCParameters
from notebooks_preprocessing_and_dataframe_creation import preprocessing_functions

In [2]:
cmap='seismic'
sns.set_palette(sns.color_palette(cmap))

In [3]:
metric = 'AUC' # F1 or AUC

In [4]:
df = pd.read_pickle('../data/unified_dataframe/data_unprocessed.pkl')
df

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,badgeType,calories,filteredDemographicVO2Max,distance,activityType,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,scl_avg,resting_hr,sleep_duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi,heart_rate_alert,startTime,endTime,label_ttm_stage,label_breq_self_determination,label_sema_negative_feelings,label_ipip_extraversion_category,label_ipip_agreeableness_category,label_ipip_conscientiousness_category,label_ipip_stability_category,label_ipip_intellect_category,label_stai_stress_category,label_panas_negative_affect
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,NaN,14.8,78.0,0.833333,0.675,0.866667,-1.788325,None,2351.59,62.79210,6517.5,[Walk],71.701565,149.0,24.0,33.0,713.0,False,NaN,62.073070,31260000.0,0.0,445.0,76.0,0.0,93.0,1.243243,0.987013,0.921642,1.341772,8833.0,83.0,1349.0,0.0,0.0,<30,MALE,<19,None,2021-05-24T00:40:00.000,2021-05-24T09:21:00.000,Maintenance,intrinsic_regulation,NaN,LOW,LOW,HIGH,HIGH,AVERAGE,NaN,NaN
1,621e328667b776a240281372,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,2619.85,41.38447,6568.9,[Walk],83.919698,132.0,7.0,30.0,1271.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8550.0,278.0,766.0,29.0,1.0,>=30,MALE,>=30,None,NaN,NaN,Maintenance,intrinsic_regulation,NaN,HIGH,AVERAGE,HIGH,LOW,LOW,NaN,NaN
2,621e326767b776a24012e179,2021-05-24,NaN,46.120,53.968,95.3,14.4,80.0,0.666667,0.925,0.766667,-4.129593,None,2204.50,52.76058,11283.7,[Walk],68.275766,278.0,41.0,67.0,616.0,False,NaN,52.516339,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16992.0,131.0,1225.0,0.0,0.0,<30,FEMALE,<19,None,NaN,NaN,Maintenance,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,621e332267b776a24092a584,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,3792.13,53.53183,16776.8,[Walk],78.842893,190.0,79.0,108.0,1063.0,False,NaN,60.977217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21284.0,175.0,1130.0,9.0,0.0,<30,MALE,21.0,None,NaN,NaN,Maintenance,identified_regulation,NaN,LOW,LOW,LOW,AVERAGE,HIGH,NaN,NaN
4,621e333567b776a240a0c217,2021-05-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[LIFETIME_DISTANCE],NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<30,MALE,21.0,None,NaN,NaN,Contemplation,intrinsic_regulation,NaN,AVERAGE,HIGH,AVERAGE,LOW,LOW,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7799,621e30b267b776a240c5e13f,2022-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,1554.15,42.17696,209.4,None,89.131283,33.0,0.0,0.0,1407.0,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287.0,19.0,126.0,0.0,0.0,<30,FEMALE,21.0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7800,621e312a67b776a240164d59,2022-01-17,34.237581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.165995,[DAILY_FLOORS],1602.67,47.22494,3806.9,None,67.473381,125.0,27.0,5.0,116.0,False,NaN,58.157746,25800000.0,0.0,379.0,51.0,1.0,94.0,0.835443,0.728571,0.796667,0.902439,4966.0,114.0,563.0,0.0,0.0,>=30,MALE,25.0,None,2022-01-17T00:45:00.000,2022-01-17T07:55:00.000,Contemplation,intrinsic_regulation,NaN,AVERAGE,LOW,HIGH,AVERAGE,LOW,NaN,NaN
7801,621e309b67b776a240b532b0,2022-01-17,31.815133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.999818,[DAILY_FLOORS],2887.52,49.82090,9000.0,"[Sport, Walk]",74.452687,164.0,37.0,63.0,875.0,False,NaN,64.007904,18060000.0,0.0,276.0,25.0,0.0,99.0,0.708333,0.609756,0.714286,1.066667,12096.0,129.0

In [5]:
df_pre = pd.read_pickle('../data/unified_dataframe/data_preprocessed.pkl')
df_pre

,id,date,nightly_temperature,nremhr,rmssd,spo2,full_sleep_breathing_rate,stress_score,sleep_points_percentage,exertion_points_percentage,responsiveness_points_percentage,daily_temperature_variation,calories,filteredDemographicVO2Max,distance,bpm,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes,mindfulness_session,scl_avg,resting_hr,sleep_duration,minutesToFallAsleep,minutesAsleep,minutesAwake,minutesAfterWakeup,sleep_efficiency,sleep_deep_ratio,sleep_wake_ratio,sleep_light_ratio,sleep_rem_ratio,steps,minutes_in_default_zone_1,minutes_below_default_zone_1,minutes_in_default_zone_2,minutes_in_default_zone_3,age,gender,bmi,label_ttm_stage,label_breq_self_determination,label_sema_negative_feelings,label_ipip_extraversion_category,label_ipip_agreeableness_category,label_ipip_conscientiousness_category,label_ipip_stability_category,label_ipip_intellect_category,label_stai_stress_category,label_panas_negative_affect,month_sin,weekday_sin,week_sin,day_sin,month_cos,weekday_cos,week_cos,day_cos,DAILY_FLOORS,DAILY_STEPS,GOAL_BASED_WEIGHT_LOSS,LIFETIME_DISTANCE,LIFETIME_FLOORS,LIFETIME_WEIGHT_GOAL_SETUP,Aerobic Workout,Bike,Bootcamp,Circuit Training,Elliptical,Hike,Interval Workout,Martial Arts,Run,Spinning,Sport,Swim,Treadmill,Walk,Weights,Workout,Yoga/Pilates,wear_day,early_features,used_during_night,different_activity_types,different_badge_types,is_index,iv_index,sri_index,sjl_index,mode_startTime,mode_endTime,mode_startTime_sin,mode_startTime_cos,mode_endTime_sin,mode_endTime_cos,isp_index,steps_is_index,steps_iv_index,steps_sri_index,steps_isp_index,exercise_is_index,exercise_iv_index,exercise_sri_index,stress_quantile,user_stress_quantile,average_sleep_duration,average_steps,is_weekend,is_holiday,startDay_sin,startDay_cos,startWeek_sin,startWeek_cos,startWeekday_sin,startWeekday_cos,startMonth_sin,startMonth_cos,startYear_sin,startYear_cos,endDay_sin,endDay_cos,endWeek_sin,endWeek_cos,endWeekday_sin,endWeekday_cos,endMonth_sin,endMonth_cos,endYear_sin,endYear_cos,startHour_sin,startHour_cos,endHour_sin,endHour_cos,Steps_hour0,Steps_hour1,Steps_hour2,Steps_hour3,Steps_hour4,Steps_hour5,Steps_hour6,Steps_hour7,Steps_hour8,Steps_hour9,Steps_hour10,Steps_hour11,Steps_hour12,Steps_hour13,Steps_hour14,Steps_hour15,Steps_hour16,Steps_hour17,Steps_hour18,Steps_hour19,Steps_hour20,Steps_hour21,Steps_hour22,Steps_hour23
0,621e2e8e67b776a24055b564,2021-05-24,34.137687,57.432,89.603,95.9,14.8,78.0,0.833333,0.675,0.866667,-1.788325,2351.59,62.79210,6517.5,71.701565,149.0,24.0,33.0,713.0,0.0,10.907065,62.073070,31260000.0,0.0,445.0,76.0,0.0,93.0,1.243243,0.987013,0.921642,1.341772,8833.0,83.0,1349.0,0.0,0.0,0.0,0.0,Underweight,Maintenance,intrinsic_regulation,NaN,LOW,LOW,HIGH,HIGH,AVERAGE,Below average,Below average,-0.866025,0.000000,3.090170e-01,-0.988468,0.5,1.000000,0.951057,0.151428,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1,0,0.873563,1,4,0.930443,0.343581,90.495130,1.091284,23.0,9.0,0.382683,-0.92388,-3.826834e-01,-0.92388,0.945035,0.287117,1.655374,89.963574,0.381108,0.224056,2.003963,77.127735,2,1,3.255517e+07,8892.885057,0.0,0.0,0.060330,0.998178,-0.281733,0.959493,0.000000,1.000000,0.012703,0.999919,-0.908669,0.417518,0.060330,0.998178,-0.0,1.0,0.000000,1.000000,0.012703,0.999919,-0.908669,0.417518,0.000000,1.000000,0.022699,0.999742,134.0,0.0,0.0,15.0,0.0,0.0,39.0,0.0,7.0,10.0,2626.0,992.0,429.0,29.0,191.0,33.0,342.0,1712.0,1838.0,160.0,155.0,37.0,31.0,53.0
1,621e2e8e67b776a24055b564,2021-05-25,33.794544,57.681,94.303,95.9,15.8,80.0,0.833333,0.725,0.866667,-2.462709,2332.08,62.67912,7178.6,70.579300,132.0,25.0,31.0,704.0,0.0,10.907065,62.121476,32880000.0,0.0,460.0,88.0,0.0,94.0,1.466667,1.142857,0.947566,1.197531,9727.0,56.0,1374.0,4.0,0.0,0.0,0.0,Underweight,Maintenance,intrinsic_regulation,1.0,LOW,LOW,HIGH,HIGH,AVERAGE,Below average,Below average,-0.866025,0.781831,3.090170e-01,-0.937752,0.5,0.623490,0.951057,0.347305,0.0,0.0,0.0,0.0,0.0,0.0,0

# Predict personality stability

In [6]:
stability = df.drop(columns=['date','startTime', 'endTime', 'label_sema_negative_feelings', 'label_ipip_agreeableness_category', 'label_ipip_extraversion_category','label_ipip_intellect_category', 'label_ipip_conscientiousness_category', 'label_stai_stress_category', 'label_panas_negative_affect', 'label_ttm_stage', 'label_breq_self_determination'])

In [7]:
train_data, test_data = preprocessing_functions.train_test_split_per_user(stability)
fold_groups = train_data.id
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

In [8]:
s = setup(data=train_data, target='label_ipip_stability_category', silent=True, session_id=123,
          fold_strategy='groupkfold', fold=3,  fold_groups=fold_groups, test_data=test_data)

,Description,Value
0,session_id,123
1,Target,label_ipip_stability_category
2,Target Type,Multiclass
3,Label Encoded,"AVERAGE: 0, HIGH: 1, LOW: 2"
4,Original Data,"(5199, 43)"
5,Missing Values,True
6,Numeric Features,33
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


In [9]:
best = compare_models(sort = '%s'%metric) # change to F1 and AUC

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.3284,0.5000,0.3333,0.1437,0.1923,0.0000,0.0000,0.0533
lda,Linear Discriminant Analysis,0.3666,0.4989,0.2596,0.4578,0.4004,-0.0071,-0.0106,0.0700
ada,Ada Boost Classifier,0.3677,0.4940,0.2944,0.4718,0.3737,0.0276,0.0325,0.1567
knn,K Neighbors Classifier,0.3247,0.4857,0.3246,0.4863,0.3415,-0.0054,-0.0020,0.7567
qda,Quadratic Discriminant Analysis,0.2700,0.4599,0.2298,0.2545,0.2484,-0.0359,-0.0618,0.0633
dt,Decision Tree Classifier,0.2503,0.4359,0.2166,0.3812,0.2797,-0.1142,-0.1269,0.0633
lr,Logistic Regression,0.3622,0.4246,0.2909,0.2471,0.2823,-0.0849,-0.0898,1.0333
lightgbm,Light Gradient Boosting Machine,0.2744,0.4079,0.2326,0.3926,0.2983,-0.0699,-0.0845,0.4067
nb,Naive Bayes,0.1592,0.3613,0.1914,0.2555,0.1618,-0.1424,-0.2026,0.7000
gbc,Gradient Boosting Classifier,0.2333,0.3601,0.1937,0.3359,0.2436,-0.1182,-0.1447,1.2000


In [10]:
lr = create_model('lr', fold=3)
tuned_lr = tune_model(lr)
d=pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(tuned_lr.coef_[0])}).sort_values(by='Value', ascending=False)
d.reset_index(inplace=True)
d=d.head(15)
d = d.drop(columns='index')
d['Label'] = 'stability'
d = d[['Label', 'Feature', 'Value']]
d.to_pickle('../data/feature_importances/stability2.pkl')
d

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6664,0.5220,0.3333,0.4441,0.5330,0.0000,0.0000
1,0.1233,0.3984,0.3333,0.0152,0.0271,0.0000,0.0000
2,0.2971,0.3533,0.2060,0.2820,0.2870,-0.2546,-0.2695
Mean,0.3622,0.4246,0.2909,0.2471,0.2823,-0.0849,-0.0898
Std,0.2265,0.0713,0.0600,0.1768,0.2066,0.1200,0.1271


,Label,Feature,Value
0,stability,steps,2.907985e-05
1,stability,distance,1.959520e-05
2,stability,minutes_below_default_zone_1,1.428257e-06
3,stability,calories,1.192105e-06
4,stability,lightly_active_minutes,6.582996e-07
5,stability,minutes_in_default_zone_1,3.503180e-07
6,stability,sedentary_minutes,1.793153e-07
7,stability,minutesAsleep,9.475818e-08
8,stability,minutesAwake,8.513596e-08
9,stability,rmssd,7.791176e-08


In [11]:
model = create_model('ada')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3610,0.4569,0.2864,0.5525,0.3862,-0.0110,-0.0139
1,0.2671,0.3418,0.1847,0.2252,0.2444,-0.1079,-0.1283
2,0.4750,0.6832,0.4121,0.6378,0.4904,0.2018,0.2395
Mean,0.3677,0.4940,0.2944,0.4718,0.3737,0.0276,0.0325
Std,0.0850,0.1418,0.0930,0.1778,0.1008,0.1294,0.1537


In [12]:
stability = df_pre.drop(columns=['date', 'label_sema_negative_feelings', 'label_ipip_agreeableness_category', 'label_ipip_extraversion_category','label_ipip_intellect_category', 'label_ipip_conscientiousness_category', 'label_stai_stress_category', 'label_panas_negative_affect', 'label_ttm_stage', 'label_breq_self_determination'])

In [13]:
train_data, test_data = preprocessing_functions.train_test_split_per_user(stability)
fold_groups = train_data.id
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

In [14]:
s = setup(data=train_data, target='label_ipip_stability_category', silent=True, session_id=123,
          fold_strategy='groupkfold', fold=3,  fold_groups=fold_groups, test_data=test_data,
          normalize=True, normalize_method='minmax', fix_imbalance=True, remove_multicollinearity = True, multicollinearity_threshold = 0.6)

,Description,Value
0,session_id,123
1,Target,label_ipip_stability_category
2,Target Type,Multiclass
3,Label Encoded,"AVERAGE: 0, HIGH: 1, LOW: 2"
4,Original Data,"(5199, 148)"
5,Missing Values,True
6,Numeric Features,107
7,Categorical Features,40
8,Ordinal Features,False
9,High Cardinality Features,False


In [15]:
best = compare_models(sort = '%s'%metric) # change to F1 and AUC

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
dummy,Dummy Classifier,0.4566,0.5000,0.3333,0.2653,0.3287,0.0000,0.0000,0.1367
nb,Naive Bayes,0.3308,0.4710,0.2873,0.4377,0.3545,-0.0263,-0.0285,0.1433
lr,Logistic Regression,0.2106,0.4584,0.1657,0.4437,0.2414,-0.1550,-0.1995,0.2767
et,Extra Trees Classifier,0.1830,0.4563,0.1594,0.2927,0.1896,-0.1143,-0.2088,0.2100
knn,K Neighbors Classifier,0.2439,0.4516,0.1809,0.3626,0.2818,-0.1053,-0.1396,0.4300
dt,Decision Tree Classifier,0.1688,0.4011,0.2310,0.1956,0.1378,-0.1520,-0.2282,0.1467
qda,Quadratic Discriminant Analysis,0.1489,0.3902,0.1162,0.2503,0.1645,-0.1957,-0.2861,0.1567
gbc,Gradient Boosting Classifier,0.1769,0.3760,0.2202,0.2246,0.1642,-0.1925,-0.2106,3.6800
lightgbm,Light Gradient Boosting Machine,0.2109,0.3684,0.2711,0.3337,0.1958,-0.0686,-0.0610,0.6200
rf,Random Forest Classifier,0.1046,0.3636,0.1128,0.0871,0.0843,-0.1842,-0.3066,0.2233


In [16]:
lr = create_model('lr', fold=3)
tuned_lr = tune_model(lr)
d=pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(tuned_lr.coef_[0])}).sort_values(by='Value', ascending=False)
d.reset_index(inplace=True)
d=d.head(15)
d = d.drop(columns='index')
d['Label'] = 'stability'
d = d[['Label', 'Feature', 'Value']]
d.to_pickle('../data/feature_importances/stability1.pkl')
d

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3603,0.5975,0.2602,0.5172,0.3996,-0.0475,-0.0547
1,0.0587,0.3532,0.0414,0.4679,0.0720,-0.2572,-0.3663
2,0.2566,0.4418,0.2082,0.3605,0.2923,-0.1520,-0.1632
Mean,0.2252,0.4642,0.1699,0.4485,0.2546,-0.1522,-0.1947
Std,0.1251,0.1010,0.0933,0.0654,0.1364,0.0856,0.1291


,Label,Feature,Value
0,stability,different_activity_types_1,11.569388
1,stability,age_0.0,5.757418
2,stability,exercise_is_index,4.562661
3,stability,steps_iv_index,4.216566
4,stability,mode_endTime_6.0,4.112594
5,stability,different_badge_types_2,3.920893
6,stability,different_activity_types_7,3.839099
7,stability,mode_startTime_2.0,3.663740
8,stability,mode_startTime_22.0,3.616961
9,stability,different_badge_types_4,3.428218


In [17]:
model = create_model('et')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.2461,0.5136,0.2188,0.5469,0.2727,-0.0207,-0.0286
1,0.0000,0.4413,0.0000,0.0000,0.0000,-0.2038,-0.4633
2,0.3029,0.4141,0.2594,0.3311,0.2960,-0.1185,-0.1343
Mean,0.1830,0.4563,0.1594,0.2927,0.1896,-0.1143,-0.2088
Std,0.1315,0.0420,0.1139,0.2249,0.1344,0.0748,0.1851


In [18]:
df = pd.read_pickle('../data/unified_dataframe/data_preprocessed.pkl')
stability = df.drop(columns=['date', 'label_sema_negative_feelings', 'label_ipip_agreeableness_category', 'label_ipip_extraversion_category','label_ipip_intellect_category', 'label_ipip_conscientiousness_category', 'label_stai_stress_category', 'label_panas_negative_affect', 'label_ttm_stage', 'label_breq_self_determination'])
stability = stability[stability['label_ipip_stability_category'].notna()]
y = stability['label_ipip_stability_category']

In [19]:
cols = ['stress_score', 'steps', 'sleep_duration']
for col in cols:
    extracted_features = extract_features(df, column_id="id", column_sort="date",
                                          n_jobs=5, column_kind=None, column_value=col,
                                          show_warnings=False)
    extracted_features = extracted_features.reset_index()
    extracted_features = extracted_features.rename(columns={"index": "id"})
    stability = stability.merge(extracted_features, how='left', on=['id'])

Feature Extraction: 100%|██████████████████████████████████████████████████████████████| 24/24 [00:03<00:00,  6.74it/s]


In [20]:
train_data, test_data = preprocessing_functions.train_test_split_per_user(stability)
fold_groups = train_data.id
train_data = train_data.drop(columns=['id'])
test_data = test_data.drop(columns=['id'])

In [21]:
s = setup(data=train_data, target='label_ipip_stability_category', silent=True, session_id=123,
          fold_strategy='groupkfold', fold=3,  fold_groups=fold_groups, test_data=test_data,
          normalize=True, normalize_method='minmax', fix_imbalance=True, remove_multicollinearity = True, multicollinearity_threshold = 0.6)

,Description,Value
0,session_id,123
1,Target,label_ipip_stability_category
2,Target Type,Multiclass
3,Label Encoded,"AVERAGE: 0, HIGH: 1, LOW: 2"
4,Original Data,"(3985, 2515)"
5,Missing Values,True
6,Numeric Features,2240
7,Categorical Features,274
8,Ordinal Features,False
9,High Cardinality Features,False


In [22]:
best = compare_models(sort = '%s'%metric) # change to F1 and AUC

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.3642,0.5908,0.3969,0.4823,0.2818,0.0708,0.1090,0.4267
nb,Naive Bayes,0.3676,0.5737,0.3816,0.4006,0.3478,0.0211,0.0494,0.3567
lightgbm,Light Gradient Boosting Machine,0.3280,0.5724,0.2644,0.4366,0.3156,-0.1239,-0.1460,0.9667
ada,Ada Boost Classifier,0.3826,0.5454,0.3645,0.3891,0.3438,-0.0202,-0.0350,0.7700
et,Extra Trees Classifier,0.3221,0.5207,0.3174,0.3305,0.2704,-0.0542,-0.0589,0.4100
gbc,Gradient Boosting Classifier,0.3726,0.5084,0.3789,0.3349,0.3204,-0.0042,-0.0085,5.5367
knn,K Neighbors Classifier,0.3112,0.5040,0.2959,0.4796,0.3238,0.0173,0.0171,1.6867
dummy,Dummy Classifier,0.4629,0.5000,0.3333,0.2437,0.3130,0.0000,0.0000,0.3333
qda,Quadratic Discriminant Analysis,0.2415,0.4909,0.3782,0.0902,0.1261,-0.0065,-0.0089,0.4600
dt,Decision Tree Classifier,0.3021,0.4597,0.3284,0.2822,0.2707,-0.0814,-0.1026,0.3567


In [23]:
lr = create_model('lr', fold=3)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.2223,0.4011,0.2853,0.2375,0.2199,-0.2166,-0.2244
1,0.2324,0.5087,0.3333,0.0540,0.0877,0.0000,0.0000
2,0.2855,0.4674,0.2466,0.4036,0.2906,-0.1164,-0.1503
Mean,0.2467,0.4591,0.2884,0.2317,0.1994,-0.1110,-0.1249
Std,0.0277,0.0443,0.0355,0.1428,0.0841,0.0885,0.0933


In [24]:
tuned_lr = tune_model(lr)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.2565,0.4077,0.3111,0.2761,0.2566,-0.1674,-0.1729
1,0.2324,0.4951,0.3333,0.0540,0.0877,0.0000,0.0000
2,0.2803,0.4718,0.2439,0.3980,0.2843,-0.1208,-0.1570
Mean,0.2564,0.4582,0.2961,0.2427,0.2095,-0.0961,-0.1100
Std,0.0195,0.0370,0.0380,0.1424,0.0869,0.0705,0.0780


In [25]:
print(tuned_lr)

LogisticRegression(C=0.049, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=123, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)


In [26]:
d=pd.DataFrame({'Feature': get_config('X_train').columns, 'Value' : abs(tuned_lr.coef_[0])}).sort_values(by='Value', ascending=False)

In [27]:
d.reset_index(inplace=True)
d=d.head(20)
d = d.drop(columns='index')
d['Label'] = 'stability'
d = d[['Label', 'Feature', 'Value']]
d.to_pickle('../data/feature_importances/stability.pkl')
d

,Label,Feature,Value
0,stability,steps__fft_coefficient__attr_angle__coeff_18,0.535124
1,stability,different_activity_types_1,0.499540
2,stability,sleep_duration__fft_coefficient__attr_angle__c...,0.488819
3,stability,steps__fft_coefficient__attr_angle__coeff_2,0.429065
4,stability,sleep_duration__symmetry_looking__r_0.05_1.0,0.417176
5,stability,age_0.0,0.392703
6,stability,sleep_duration__autocorrelation__lag_9,0.377075
7,stability,sleep_duration__fft_coefficient__attr_angle__c...,0.364308
8,stability,steps__cwt_coefficients__coeff_14__w_2__widths...,0.360312
9,stability,steps__agg_linear_trend__attr_stderr__chunk_le...,0.355880
